In [ ]:
import os
import phoenix as px
from llama_index.core import (
    Settings,
    VectorStoreIndex,
    SimpleDirectoryReader,
    set_global_handler,
)
from llama_index.embeddings.azure_openai import AzureOpenAIEmbedding
from llama_index.llms.azure_openai import AzureOpenAI


In [ ]:
from dotenv import load_dotenv
load_dotenv()

# The base URL for your Azure OpenAI resource. e.g. "https://<your resource name>.openai.azure.com"
base_url = os.getenv("AZURE_OPENAI_ENDPOINT")
# The API key for your Azure OpenAI resource.
api_key = os.getenv("AZURE_OPENAI_API_KEY")
# Setting up the deployment name
gpt_model_name = os.getenv("AZURE_OPENAI_GPT_MODEL")
embedding_model_name = os.getenv("AZURE_OPENAI_EMBEDDING_MODEL")
# Currently Chat Completions API have the following versions available: 2023-03-15-preview
api_version = os.getenv("AZURE_OPENAI_API_VERSION")
api_type = "azure"

In [ ]:
# To view traces in Phoenix, you will first have to start a Phoenix server. You can do this by running the following:
session = px.launch_app()


# Once you have started a Phoenix server, you can start your LlamaIndex application and configure it to send traces to Phoenix. To do this, you will have to add configure Phoenix as the global handler

set_global_handler("arize_phoenix")

llm=AzureOpenAI(
            azure_endpoint=base_url,
            api_key=api_key,
            deployment_name=gpt_model_name,
            model=gpt_model_name,
            api_version=api_version,
)
# LlamaIndex application initialization may vary
# depending on your application
Settings.llm = llm
Settings.embed_model = AzureOpenAIEmbedding(
    azure_endpoint=base_url,
    api_key=api_key,
    deployment_name=embedding_model_name,
    model=embedding_model_name,
    api_version=api_version,
    )




In [ ]:
# Load your data and create an index. Note you usually want to store your index in a persistent store like a database or the file system
documents = SimpleDirectoryReader("data").load_data()
index = VectorStoreIndex.from_documents(documents)

query_engine = index.as_query_engine()

# Query your LlamaIndex application
query_engine.query("What is the meaning of life?")
query_engine.query("Why did the cow jump over the moon?")

# View the traces in the Phoenix UI
px.active_session().url